## Import das bibliotecas

In [1]:
!pip install webrtcvad
!pip install scipy
!pip install pydub
!pip install librosa

You should consider upgrading via the 'c:\users\csorgo\pycharmprojects\miau\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\csorgo\pycharmprojects\miau\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\csorgo\pycharmprojects\miau\venv\scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\csorgo\pycharmprojects\miau\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import wave
import librosa
import webrtcvad
import contextlib
from os import listdir
from scipy.io import wavfile
import IPython.display as ipd
from pydub import AudioSegment
from os.path import isfile, join

In [5]:
vad = webrtcvad.Vad()

In [3]:
audio_path = '../../data/raw/youtube_audios/'
onlyfiles = [f for f in listdir(audio_path) if isfile(join(audio_path, f))]

In [5]:
class Frame(object):
    """
    Representa um 'quadro' de dados de audio. Um quadro é um pedaço em millisegundos de um arquivo de audio.
    """
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration

In [4]:
def read_wave(path):
    """
    Lê um arquivo .wav.
    Recebe o caminho e devolve as informações de audio do PCM (Pulse-Code Modulation) e o sample rate.
    """
    with contextlib.closing(wave.open(path, 'rb')) as wav_file:
        num_channels = wav_file.getnchannels()
        assert num_channels == 1
        sample_width = wav_file.getsampwidth()
        assert sample_width == 2
        sample_rate = wav_file.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wav_file.readframes(wav_file.getnframes())
        return pcm_data, sample_rate

In [ ]:
def frame_generator(frame_duration_ms, pcm_audio_data, sample_rate):
    """
    Gera quadros de audio usando PCM audio data.
    Recebe a duração desejada em millisegundos, os dados do PCM, e o
    sample rate.
    Produz uma série de audios (Yield) com a duração fornecida.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n

In [ ]:
def vad_collector(sample_rate, frame_duration_ms, padding_duration_ms, vad, frames):
    
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])

            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

In [6]:
for file in onlyfiles:
    print(file)
    sample_rate, data = wavfile.read(f'{audio_path}{file}')
    # x, sample_rate = librosa.load(f'{audio_path}{file}')
    print(sample_rate)


-gSfPQqi6nI.wav
c:\Users\Csorgo\PycharmProjects\miau\venv\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
22050
-uGHAvfqs2I.wav
c:\Users\Csorgo\PycharmProjects\miau\venv\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
22050
-vZ-_4oFCzs.wav
22050
-zCtwbk005g.wav
22050
-ZJqu_4zLMc.wav


KeyboardInterrupt: 